In [1]:
!nvidia-smi

Sat Jun 12 19:33:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#https://drive.google.com/file/d/1EVwP6MUQAeSnGM-ywBRHspCn_OWEfzJN/view?usp=sharing
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1EVwP6MUQAeSnGM-ywBRHspCn_OWEfzJN" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1EVwP6MUQAeSnGM-ywBRHspCn_OWEfzJN" -o train.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    259      0 --:--:--  0:00:01 --:--:--   259
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
100  955k  100  955k    0     0   389k      0  0:00:02  0:00:02 --:--:--  389k


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('train.csv')
df.head()

,id,time,x,y
0,1,0,800,0
1,1,780,780,0
2,1,1572,792,0
3,1,2392,820,0
4,1,3196,804,0


In [4]:
df['marker'] = np.multiply(df[['id','y']].groupby('id').agg(np.cumsum).values.ravel(), df['y'].values)
df['start'] = ((df['y'].shift(1, fill_value=0) == 0).values & (df['y'] == 1)).values
df['end'] = ((df['y'].shift(-1, fill_value=0) == 0).values & (df['y'] == 1)).values

q = df.loc[df['end'],'marker'] - df.loc[df['end'],'start']
print('аномалий: ', q.shape[0], 'средняя длина: ', int(q.mean()*100.0)*0.01, 'min: ', q.min(), 'max: ', q.max())

аномалий:  791 средняя длина:  25.060000000000002 min:  6 max:  81


In [5]:
import plotly.express as px
import plotly.graph_objects as go

for q in [163]:
    t = df.loc[df['id'] == q].sort_values('time').reset_index(drop=True)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=t['time'], y=t['x'] - t['x'].mean(),
                        mode='lines',
                        name='lines'))
    qt = t.loc[t.y==1].reset_index(drop=True)
    fig.add_trace(go.Scatter(x=qt['time'], y=qt['x'] - t['x'].mean(),
                        mode='markers', name='markers'))
    print(t.x.mean())
    fig.show()


728.0415584415584


In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import torch
from torch.utils.data import Dataset

def scale_ts(vls):
    sc = RobustScaler()
    return sc.fit_transform(vls.reshape(-1,1)).ravel()
def scale_sts(vls):
    sc = StandardScaler()
    return sc.fit_transform(vls.reshape(-1,1)).ravel()
def scale_time_ts(vls):
    sc = MinMaxScaler()
    return sc.fit_transform(vls.reshape(-1,1)).ravel()

class CardioDataset(Dataset):
    def __init__(self, df, win_size=33):
        self.df = df.sort_values(['id','time']).reset_index(drop=True).copy()
        self.df['time'] = df.groupby('id')['time'].agg('diff').fillna(0).values
        self.df['time'] = scale_time_ts(self.df['time'].values)
        self.df['x'] = scale_ts(self.df['x'].values)
        self.win_size = win_size

        self.start_index = []
        self.end_index = []

        total_len = 0
        for q,qdf in self.df.groupby('id'):
            #qdf['x'] = scale_ts(qdf['x'].values)
            for i in range(qdf.shape[0]):
                self.start_index.append(max(total_len, total_len + i - win_size // 2))
                self.end_index.append(min(total_len + qdf.shape[0], total_len + i + win_size // 2))
            total_len += qdf.shape[0]

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        i0 = self.start_index[idx]
        i1 = self.end_index[idx]

        x_mat = np.zeros((self.win_size,1))
        s = i1 - i0
        x_mat[-s:,0] = scale_ts(self.df.iloc[i0:i1].x.values)
        #x_mat[-s:,0] = self.df.iloc[i0:i1].x.values
        
        return {"x": x_mat,
                "y": [self.df.y.values[idx]],
                "start": i0,
                "end": i1
               }

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [8]:
from sklearn.metrics import roc_curve, precision_recall_curve, f1_score

def threshold_search(y_true, y_proba):
    precision , recall, thresholds = precision_recall_curve(y_true, y_proba)
    thresholds = np.append(thresholds, 1.001) 
    F = 2 / (1/precision + 1/recall)
    best_score = np.max(F)
    best_th = thresholds[np.argmax(F)]
    return best_th , best_score

In [9]:
%%writefile lookahead.py
# Lookahead implementation from https://github.com/rwightman/pytorch-image-models/blob/master/timm/optim/lookahead.py

""" Lookahead Optimizer Wrapper.
Implementation modified from: https://github.com/alphadl/lookahead.pytorch
Paper: `Lookahead Optimizer: k steps forward, 1 step back` - https://arxiv.org/abs/1907.08610
"""
import torch
from torch.optim.optimizer import Optimizer
from collections import defaultdict

class Lookahead(Optimizer):
    def __init__(self, base_optimizer, alpha=0.5, k=6):
        if not 0.0 <= alpha <= 1.0:
            raise ValueError(f'Invalid slow update rate: {alpha}')
        if not 1 <= k:
            raise ValueError(f'Invalid lookahead steps: {k}')
        defaults = dict(lookahead_alpha=alpha, lookahead_k=k, lookahead_step=0)
        self.base_optimizer = base_optimizer
        self.param_groups = self.base_optimizer.param_groups
        self.defaults = base_optimizer.defaults
        self.defaults.update(defaults)
        self.state = defaultdict(dict)
        # manually add our defaults to the param groups
        for name, default in defaults.items():
            for group in self.param_groups:
                group.setdefault(name, default)

    def update_slow(self, group):
        for fast_p in group["params"]:
            if fast_p.grad is None:
                continue
            param_state = self.state[fast_p]
            if 'slow_buffer' not in param_state:
                param_state['slow_buffer'] = torch.empty_like(fast_p.data)
                param_state['slow_buffer'].copy_(fast_p.data)
            slow = param_state['slow_buffer']
            slow.add_(group['lookahead_alpha'], fast_p.data - slow)
            fast_p.data.copy_(slow)

    def sync_lookahead(self):
        for group in self.param_groups:
            self.update_slow(group)

    def step(self, closure=None):
        # print(self.k)
        #assert id(self.param_groups) == id(self.base_optimizer.param_groups)
        loss = self.base_optimizer.step(closure)
        for group in self.param_groups:
            group['lookahead_step'] += 1
            if group['lookahead_step'] % group['lookahead_k'] == 0:
                self.update_slow(group)
        return loss

    def state_dict(self):
        fast_state_dict = self.base_optimizer.state_dict()
        slow_state = {
            (id(k) if isinstance(k, torch.Tensor) else k): v
            for k, v in self.state.items()
        }
        fast_state = fast_state_dict['state']
        param_groups = fast_state_dict['param_groups']
        return {
            'state': fast_state,
            'slow_state': slow_state,
            'param_groups': param_groups,
        }

    def load_state_dict(self, state_dict):
        fast_state_dict = {
            'state': state_dict['state'],
            'param_groups': state_dict['param_groups'],
        }
        self.base_optimizer.load_state_dict(fast_state_dict)

        # We want to restore the slow state, but share param_groups reference
        # with base_optimizer. This is a bit redundant but least code
        slow_state_new = False
        if 'slow_state' not in state_dict:
            print('Loading state_dict from optimizer without Lookahead applied.')
            state_dict['slow_state'] = defaultdict(dict)
            slow_state_new = True
        slow_state_dict = {
            'state': state_dict['slow_state'],
            'param_groups': state_dict['param_groups'],  # this is pointless but saves code
        }
        super(Lookahead, self).load_state_dict(slow_state_dict)
        self.param_groups = self.base_optimizer.param_groups  # make both ref same container
        if slow_state_new:
            # reapply defaults to catch missing lookahead specific ones
            for name, default in self.defaults.items():
                for group in self.param_groups:
                    group.setdefault(name, default)

Overwriting lookahead.py


In [10]:
%%writefile ralamb.py
import torch, math
from torch.optim.optimizer import Optimizer

# RAdam + LARS
class Ralamb(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(Ralamb, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(Ralamb, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('Ralamb does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                # Decay the first and second moment running average coefficient
                # m_t
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                # v_t
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]

                if state['step'] == buffered[0]:
                    N_sma, radam_step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        radam_step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        radam_step_size = 1.0 / (1 - beta1 ** state['step'])
                    buffered[2] = radam_step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                radam_step = p_data_fp32.clone()
                if N_sma >= 5:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    radam_step.addcdiv_(-radam_step_size * group['lr'], exp_avg, denom)
                else:
                    radam_step.add_(-radam_step_size * group['lr'], exp_avg)

                radam_norm = radam_step.pow(2).sum().sqrt()
                weight_norm = p.data.pow(2).sum().sqrt().clamp(0, 10)
                if weight_norm == 0 or radam_norm == 0:
                    trust_ratio = 1
                else:
                    trust_ratio = weight_norm / radam_norm

                state['weight_norm'] = weight_norm
                state['adam_norm'] = radam_norm
                state['trust_ratio'] = trust_ratio

                if N_sma >= 5:
                    p_data_fp32.addcdiv_(-radam_step_size * group['lr'] * trust_ratio, exp_avg, denom)
                else:
                    p_data_fp32.add_(-radam_step_size * group['lr'] * trust_ratio, exp_avg)

                p.data.copy_(p_data_fp32)

        return loss

Overwriting ralamb.py


In [11]:
from lookahead import *
from ralamb import * 

def Over9000(params, alpha=0.5, k=6, *args, **kwargs):
     ralamb = Ralamb(params, *args, **kwargs)
     return Lookahead(ralamb, alpha, k)

RangerLars = Over9000

In [12]:
class CardioCnn1(nn.Module):
    def __init__(self, win_size, output_size = 1, 
                 out_chan = 32, 
                 top_classifier_units = 16,
                 rnn_units = 16,
                 kern_sizes = [9,5,7,3]):
        super().__init__()
        self.cnn0 = nn.Conv1d(1, out_chan, kern_sizes[0])
        self.cnn1 = nn.Conv1d(32, out_chan, kern_sizes[1])
        self.cnn2 = nn.Conv1d(32, out_chan, kern_sizes[2])
        self.cnn3 = nn.Conv1d(32, out_chan, kern_sizes[3])

        self._gru = nn.GRU(input_size=1, 
                           num_layers=4,
                           dropout=0.3,
                           hidden_size=rnn_units, 
                           batch_first=True, 
                           bidirectional=True)

        self.sum_chans = 37

        self._head = nn.Sequential(
            nn.LayerNorm(self.sum_chans),
            nn.Linear(self.sum_chans, top_classifier_units),
            nn.LayerNorm(top_classifier_units),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(top_classifier_units, top_classifier_units),
            nn.LayerNorm(top_classifier_units),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(top_classifier_units, output_size),            
        ) 
        
    def forward(self, x_feats):
        encoded,_ = self._gru(x_feats)
        encoded0 = self.cnn0(x_feats.swapaxes(1,2))
        encoded1 = self.cnn1(encoded0)
        encoded2 = self.cnn2(encoded1)
        encoded3 = self.cnn3(encoded2)

        x6 = torch.max(encoded3, axis=1).values
        x6 = torch.squeeze(x6)

        x8 = torch.max(encoded, axis=1).values
        x8 = torch.squeeze(x8)

        x = torch.cat([x6,x8],-1)
        x = self._head(x)
        return x

In [13]:
class CardioCnn(nn.Module):
    def __init__(self, win_size, output_size = 1, 
                 out_chan = 32, 
                 top_classifier_units = 32,
                 rnn_units = 8,
                 kern_sizes = [3,5,7,9]):
        super().__init__()
        self.cnn0 = nn.Conv1d(1, out_chan, kern_sizes[0])
        self.cnn1 = nn.Conv1d(1, out_chan, kern_sizes[1])
        self.cnn2 = nn.Conv1d(1, out_chan, kern_sizes[2])
        self.cnn3 = nn.Conv1d(1, out_chan, kern_sizes[3])

        self._gru = nn.GRU(input_size=1, 
                           num_layers=4,
                           dropout=0.3,
                           hidden_size=rnn_units, 
                           batch_first=True, 
                           bidirectional=True)

        self.sum_chans = (win_size*4 - np.sum(kern_sizes) + 4) + 2*rnn_units

        self._head = nn.Sequential(
            nn.LayerNorm(self.sum_chans),
            nn.Linear(self.sum_chans, top_classifier_units),
            nn.LayerNorm(top_classifier_units),
            nn.Dropout(0.5),
            nn.GELU(),
            nn.Linear(top_classifier_units, top_classifier_units),
            nn.LayerNorm(top_classifier_units),
            nn.Dropout(0.2),
            nn.GELU(),
            nn.Linear(top_classifier_units, output_size),            
        ) 
        
    def forward(self, x_feats):
        encoded,_ = self._gru(x_feats)
        encoded0 = self.cnn0(x_feats.swapaxes(1,2))
        encoded1 = self.cnn1(x_feats.swapaxes(1,2))
        encoded2 = self.cnn2(x_feats.swapaxes(1,2))
        encoded3 = self.cnn3(x_feats.swapaxes(1,2))

        x0 = torch.max(encoded0, axis=1).values
        x0 = torch.squeeze(x0)
        x1 = torch.mean(encoded0, axis=1)
        x1 = torch.squeeze(x1)

        x2 = torch.max(encoded1, axis=1).values
        x2 = torch.squeeze(x2)
        x3 = torch.mean(encoded1, axis=1)
        x3 = torch.squeeze(x3)

        x4 = torch.max(encoded2, axis=1).values
        x4 = torch.squeeze(x4)
        x5 = torch.mean(encoded2, axis=1)
        x5 = torch.squeeze(x5)

        x6 = torch.max(encoded3, axis=1).values
        x6 = torch.squeeze(x6)
        x7 = torch.mean(encoded3, axis=1)
        x7 = torch.squeeze(x7)

        x8 = torch.max(encoded, axis=1).values
        x8 = torch.squeeze(x8)

        x = torch.cat([x0,x2,x4,x6,x8],-1)
        x = self._head(x)
        return x

In [ ]:
import joblib 
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from torch.utils.data import RandomSampler, SequentialSampler, DataLoader
import tqdm

WIN_SIZE = 17

kf = KFold(n_splits=5, shuffle=True, random_state=42)
ifold = 0
fold_metrics = []
for tr_id, va_id in kf.split(df['id'].unique()):
    train_df = df.loc[df['id'].isin(tr_id)].reset_index(drop=True)
    valid_df = df.loc[df['id'].isin(va_id)].reset_index(drop=True)

    train_ds = CardioDataset(train_df, WIN_SIZE)
    valid_ds = CardioDataset(valid_df, WIN_SIZE)

    train_sampler = RandomSampler(train_ds)
    valid_sampler = SequentialSampler(valid_ds)
    batch_size = 1024
    train_dl = DataLoader(train_ds, sampler=train_sampler, batch_size=batch_size, num_workers=2)
    valid_dl = DataLoader(valid_ds, sampler=valid_sampler, batch_size=batch_size, num_workers=2)

    model = CardioCnn(WIN_SIZE, 1).to(device)

    nepochs = 30

    criterion = torch.nn.BCEWithLogitsLoss()
    
    optimizer = torch.optim.AdamW(model.parameters(),
                          lr = 1e-3                          
                       )
    
    optimizer = Lookahead(optimizer, 0.5, 6)
    '''
    optimizer = RangerLars(model.parameters(), lr=1e-3)
    '''
    for epoch in range(nepochs):
        model.train();
        optimizer.zero_grad()
        i = 0
        train_losses = []
        for x in train_dl:
            out = model(x['x'].float().to(device))
            loss = criterion(out.flatten(), x['y'][0].float().to(device)) 
            loss.backward()
            optimizer.step() 
            optimizer.zero_grad()
            i += 1
        #torch.save(model.state_dict(), 'model_'+str(epoch) + '.pth')
            train_losses.append(loss.item())
        
        valid_losses = []
        valid_df_predict = np.zeros(valid_df.shape[0])
        model.eval();
        total_len = 0
        for xx in valid_dl:
            out = model(xx['x'].float().to(device))
            valid_loss = criterion(out.flatten(), xx['y'][0].float().to(device)) 
            valid_losses.append(valid_loss.item())
            out = torch.sigmoid(out.flatten()).detach().cpu().numpy()
            for j in range(xx['x'].shape[0]):
                i0 = total_len + j
                valid_df_predict[i0] = out[j]
            total_len += xx['x'].shape[0]
        y_proba = valid_df_predict
        y_true = valid_df.y.values

        best_th , best_score = threshold_search(y_true, y_proba)
        #best_th = 0.066
        #best_score = f1_score(y_true.astype(int), (y_proba > best_th).astype(int)) #, average='micro'
        print('fold', ifold, 'epoch', epoch, 'best_th', best_th, 'f1-score', best_score) #'step', i,
        print('fold', ifold, 'epoch', epoch, 'train_loss', np.mean(train_losses), 'valid_loss', np.mean(valid_losses)) 
        model.train();

    fold_metrics.append(best_score)
    ifold += 1

fold 0 epoch 0 best_th 0.15665209293365479 f1-score 0.327217125382263
fold 0 epoch 0 train_loss 0.47142459026404787 valid_loss 0.40159839527173474


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning:

divide by zero encountered in true_divide



fold 0 epoch 1 best_th 0.11991103738546371 f1-score 0.35934472432622866
fold 0 epoch 1 train_loss 0.4051912519193831 valid_loss 0.3917012092742053
fold 0 epoch 2 best_th 0.12936337292194366 f1-score 0.394963022186688
fold 0 epoch 2 train_loss 0.3834331645852044 valid_loss 0.36691246926784515
fold 0 epoch 3 best_th 0.29211464524269104 f1-score 0.610727969348659
fold 0 epoch 3 train_loss 0.3420410801966985 valid_loss 0.2908031926913695
fold 0 epoch 4 best_th 0.14546628296375275 f1-score 0.7156611039794609
fold 0 epoch 4 train_loss 0.2721460479355994 valid_loss 0.23315543613650583
fold 0 epoch 5 best_th 0.15395276248455048 f1-score 0.7381103360811667
fold 0 epoch 5 train_loss 0.2321420003260885 valid_loss 0.21233875168995422
fold 0 epoch 6 best_th 0.14171679317951202 f1-score 0.7502423263327949
fold 0 epoch 6 train_loss 0.21412794895115353 valid_loss 0.2077850733291019
fold 0 epoch 7 best_th 0.1292906254529953 f1-score 0.7557692307692309
fold 0 epoch 7 train_loss 0.20512528078896658 valid

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning:

divide by zero encountered in true_divide



fold 1 epoch 1 best_th 0.16994647681713104 f1-score 0.36209751609935603
fold 1 epoch 1 train_loss 0.4246168419336661 valid_loss 0.3271635557923998
fold 1 epoch 2 best_th 0.35895806550979614 f1-score 0.4596476853748464
fold 1 epoch 2 train_loss 0.3876115305301471 valid_loss 0.29312003191028324
fold 1 epoch 3 best_th 0.4370797574520111 f1-score 0.5977742448330684
fold 1 epoch 3 train_loss 0.33790309001237917 valid_loss 0.2492177731224469
fold 1 epoch 4 best_th 0.6244283318519592 f1-score 0.6342857142857143
fold 1 epoch 4 train_loss 0.2703602684613986 valid_loss 0.22408693337014743
fold 1 epoch 5 best_th 0.6337279081344604 f1-score 0.6414740787008121
fold 1 epoch 5 train_loss 0.23794980423572737 valid_loss 0.22317922062107495
fold 1 epoch 6 best_th 0.6553953886032104 f1-score 0.6465976795233616
fold 1 epoch 6 train_loss 0.20843807894449967 valid_loss 0.22594456600823573
fold 1 epoch 7 best_th 0.5878229141235352 f1-score 0.6434250764525994
fold 1 epoch 7 train_loss 0.19927828873579317 vali

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning:

divide by zero encountered in true_divide



fold 2 epoch 1 best_th 0.13210336863994598 f1-score 0.3330064718572269
fold 2 epoch 1 train_loss 0.4164383813738823 valid_loss 0.3334173771242301
fold 2 epoch 2 best_th 0.1374083161354065 f1-score 0.35626242544731607
fold 2 epoch 2 train_loss 0.40397013202309606 valid_loss 0.3199323272953431
fold 2 epoch 3 best_th 0.2821628451347351 f1-score 0.4460835967685282
fold 2 epoch 3 train_loss 0.38476394265890124 valid_loss 0.28761005960404873
fold 2 epoch 4 best_th 0.31215187907218933 f1-score 0.6971830985915494
fold 2 epoch 4 train_loss 0.34140932112932204 valid_loss 0.2158253025263548
fold 2 epoch 5 best_th 0.34277886152267456 f1-score 0.7633587786259541
fold 2 epoch 5 train_loss 0.2738998580724001 valid_loss 0.1631833485638102
fold 2 epoch 6 best_th 0.2162344753742218 f1-score 0.7782156932998926
fold 2 epoch 6 train_loss 0.2379907626658678 valid_loss 0.15124413060645261
fold 2 epoch 7 best_th 0.2389933168888092 f1-score 0.7847730600292826
fold 2 epoch 7 train_loss 0.2230108942836523 valid_